### Initialize

In [62]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

TRAINING_FILE = "/home/straw/Data_mining/Pandas_Matplotlib/nlp/training_handled.csv"
TESTING_FILE = "/home/straw/Data_mining/Pandas_Matplotlib/nlp/testing_handled.csv"

### Extract label array, corpus from handled file 

In [13]:
with open(TRAINING_FILE, 'r') as inp_file:
    data = inp_file.readlines()
    corpus = []
    nlp_y = []

    for line in data:
        line = line.split('\"')
        nlp_y.append(line[1])
        corpus.append(line[3])
    inp_file.close()

### Get features matrix using tf-idf
1.tf - term frequency 
2.idf - inverse document frequency 

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# convert from sparse matrix => numpy array => dataframe
nlp_X = vectorizer.fit_transform(corpus).toarray()
nlp_X = pd.DataFrame(nlp_X, columns = vectorizer.get_feature_names())

# split features matrix nlp_X, target array nlp_y using 4-fold cross validation
train_X, test_X, train_y, test_y = train_test_split(nlp_X, nlp_y, train_size = 0.75, shuffle = True, random_state = 16)

### Define each Model as an object of class Classifier Model 
#### Ps: cái này e tham khảo idol Nguyễn Minh Quang - aka Moi


In [15]:
class ClassifierModel(object):
    def __init__(self, features_train = None, labels_train = None,  estimator = None, features_test = None, labels_test = None):
        self.features_train = features_train
        self.features_test = features_test
        self.labels_train = labels_train
        self.labels_test = labels_test
        self.estimator = estimator
    def get_estimator(self):
        return self.estimator
    def train(self):
        self.estimator.fit(self.features_train, self.labels_train)
        self.get_result()
    def predict(self,input):
        return self.estimator.predict(input)
    """
        predict and retrive classificatino_report (precision, recall, accuracy, f1 score)
    """
    def get_result(self):
        y_true, y_pred = self.labels_test, self.estimator.predict(self.features_test)
        print(classification_report(y_true, y_pred))

### Naive Bayes model 

In [25]:
from sklearn.naive_bayes import GaussianNB
naive_bayes = ClassifierModel(train_X, train_y, GaussianNB(), test_X, test_y)
naive_bayes.train()

precision    recall  f1-score   support

    __CTXH__       0.61      0.70      0.65       122
      __DS__       0.76      0.76      0.76       134
      __KD__       0.85      0.75      0.80       130
      __KH__       0.69      0.73      0.71       113
      __PL__       0.87      0.80      0.83       122
      __SK__       0.87      0.81      0.84       130
      __TG__       0.85      0.91      0.88       122
      __TT__       0.97      0.94      0.95       118
      __VH__       0.77      0.87      0.82       134
      __VT__       0.89      0.77      0.82       125

    accuracy                           0.80      1250
   macro avg       0.81      0.80      0.81      1250
weighted avg       0.81      0.80      0.81      1250



In [ ]:
### RandomForest Model 
1. max_depth: độ sâu cây
2. n_estimators: số lượng cây tạo ra 
3. criterion: độ đo (Gini/Entropy)
4. random_state: em không hiểu lắm nhưng thấy cho vào thì nó tăng độ chính xác (nên để trong khoảng 0 -> 42)

In [26]:
from sklearn.ensemble import RandomForestClassifier
random_forest = ClassifierModel(train_X, train_y, RandomForestClassifier(max_depth = 230, n_estimators = 1000, criterion="entropy", random_state=16), test_X, test_y)
random_forest.train()

precision    recall  f1-score   support

    __CTXH__       0.77      0.63      0.69       122
      __DS__       0.85      0.79      0.82       134
      __KD__       0.76      0.88      0.82       130
      __KH__       0.83      0.77      0.80       113
      __PL__       0.83      0.93      0.88       122
      __SK__       0.84      0.88      0.86       130
      __TG__       0.90      0.88      0.89       122
      __TT__       0.96      0.97      0.97       118
      __VH__       0.93      0.93      0.93       134
      __VT__       0.87      0.85      0.86       125

    accuracy                           0.85      1250
   macro avg       0.85      0.85      0.85      1250
weighted avg       0.85      0.85      0.85      1250



### SVM model 

In [27]:
from sklearn.svm import SVC
SVM = ClassifierModel(train_X, train_y, SVC(), test_X, test_y)
SVM.train()

precision    recall  f1-score   support

    __CTXH__       0.70      0.82      0.75       122
      __DS__       0.89      0.86      0.87       134
      __KD__       0.91      0.91      0.91       130
      __KH__       0.78      0.92      0.85       113
      __PL__       0.96      0.91      0.93       122
      __SK__       0.93      0.84      0.88       130
      __TG__       0.93      0.92      0.92       122
      __TT__       0.98      0.95      0.97       118
      __VH__       0.97      0.92      0.94       134
      __VT__       0.97      0.94      0.95       125

    accuracy                           0.90      1250
   macro avg       0.90      0.90      0.90      1250
weighted avg       0.90      0.90      0.90      1250



### VotingClassifier Model 

In [28]:
from sklearn.ensemble import VotingClassifier
voting_classifier_hard = ClassifierModel(train_X, train_y, 
                                         VotingClassifier(estimators=[("naive", naive_bayes.get_estimator()),
                                                                    ("random_forest", random_forest.get_estimator()),
                                                                    ("svm", SVM.get_estimator())],
                                                          voting = "hard",
                                                          weights = [2, 2, 3]), 
                                         test_X, test_y)
voting_classifier_hard.train()

precision    recall  f1-score   support

    __CTXH__       0.72      0.76      0.74       122
      __DS__       0.90      0.87      0.89       134
      __KD__       0.91      0.92      0.91       130
      __KH__       0.84      0.93      0.88       113
      __PL__       0.93      0.94      0.94       122
      __SK__       0.93      0.86      0.90       130
      __TG__       0.93      0.92      0.93       122
      __TT__       0.98      0.97      0.98       118
      __VH__       0.95      0.96      0.96       134
      __VT__       0.97      0.93      0.95       125

    accuracy                           0.91      1250
   macro avg       0.91      0.91      0.91      1250
weighted avg       0.91      0.91      0.91      1250



### Confusion Matrix 

In [61]:
from sklearn.metrics import confusion_matrix

voting_conf_matrix = confusion_matrix(test_y, voting_classifier_hard.predict(test_X))
labels = ['CTXH', 'DS', 'KD', 'KH', 'PL', 'SK', 'TG', 'TT', 'VH', 'VT']
voting_conf_matrix = pd.DataFrame(voting_conf_matrix, index = labels, columns= labels)

voting_conf_matrix


,CTXH,DS,KD,KH,PL,SK,TG,TT,VH,VT
CTXH,93,3,3,6,5,5,4,0,2,1
DS,5,117,2,4,1,1,0,2,2,0
KD,7,0,119,1,0,0,1,0,0,2
KH,3,4,0,105,0,1,0,0,0,0
PL,5,0,1,0,115,0,1,0,0,0
SK,6,2,2,7,0,112,0,0,1,0
TG,4,1,2,0,1,1,112,0,1,0
TT,2,0,0,0,1,0,0,115,0,0
VH,1,3,0,0,0,0,1,0,129,0
VT,3,0,2,2,0,0,1,0,1,116


### Extract text from handled testing file 

In [65]:
with open(TESTING_FILE, 'r') as inp_file:
    nlp_testing = []
    for line in inp_file.readlines():
        nlp_testing.append(line.strip('\n'))
    inp_file.close()
nlp_testing

ất_lực tâm_lý Đồng_thời xã_hội hiện_đại đàn_ông đối_mặt công_việc bận_rộn đấu_tranh sinh_tồn uống hút chất kích_thích suy_giảm tình_dục nam_giới Anh sợ kiếm tiền Đàn_ông niềm tiền bổ_sung khiếm_khuyết Thế_là lao kiếm tiền tích_lũy rơi cảnh thanh_toán hàng đống loại_hóa_đơn Vậy thất_nghiệp kiếm tiền Anh sợ vợ Người chồng trụ_cột gia_đình vợ đứa chỗ dựa tinh_thần sống phấn_đấu tổ_ấm trở Vì_thế sợ vợ đương_nhiên xã_hội ngày_nay phụ_nữ quyền bình_đẳng nam_giới ',
 'Phát_hiện mộ hầu vua Ai Cập_Mộ cận_thần vua Aha vị vua triều_đại Ai Cập cổ_đại phát_hiện_tại Abydos Bằng_chứng kẻ hầu_cận hiến tế đi phục_vụ nhà_vua thế_giới kia Quần_thể Abydos bao_gồm nhà_nguyện nguyên_vẹn bao quanh 6 mộ cận_thần phục_vụ nhà_vua từ_giã cõi_đời Khu_vực nằm mộ vua Aha 1,5 km Ngôi mộ vị vua phát_hiện 1900 nghiên_cứu xương khảo_cổ kết_luận kẻ hầu_cận hiến tế chôn vua Aha Giám_đốc dự_án David O Conner nhận_định phát_hiện ý_nghĩa đế_chế Aha kiến_trúc hoàng_gia Hình_thức sơ_đồ khu_vực mô_hình nhà_nguyện nằm coi khuôn

### Predict testing data

In [70]:


# extract features with previously fitted TfidfVectorizer
nlp_testing_X = vectorizer.transform(nlp_testing)
nlp_testing_pred = voting_classifier_hard.predict(nlp_testing_X.toarray())
nlp_testing_pred

array(['__KD__', '__DS__', '__PL__', ..., '__SK__', '__CTXH__',
       '__CTXH__'], dtype='<U8')

### Write predicted labels to file 

In [74]:
with open("/home/straw/Data_mining/Pandas_Matplotlib/nlp/predict.txt", 'w') as out_file:
    for i in range(len(nlp_testing_pred)):
        out_file.write(nlp_testing_pred[i] + '\n')